# Week 2 Quiz: Manipulating DataFrames

**Note**: 

> This exercise has been written out in something called a Jupyter Notebook. We'll discuss Jupyter Notebooks in more detail later in this specialization—they are very a powerful tool for data science communication!—but for the time being, the notebook is just a convenient way for us to write out the exercise. You don't need to *do* anything with the notebook except read its contents—just use write your Python code in a regular `.py` file.

**WARNING:**

> When asked to round your answers to a certain number of decimals, do *not* round any results until you've finished your computations and have your final answer! For example, if you were to calculate the average hourly wage for workers, and you did so by first calculating the average weekly salary of workers and the average hours worked per week, then divided the first number by the second, you should NOT round the average weekly salary of workers or the  average hours worked per week. Rounding intermediate results can lead to compounding errors that cause problems for the autograder.

## Estimating Labor Market Returns to Education

In this exercise, we're going to use data from the [American Communities Survey (ACS)](https://usa.ipums.org/usa/acs.shtml) to study the relationship between educational attainment and wages. The ACS is a survey conducted by the United States Census Bureau (though it is not "The Census," which is a counting of every person in the United States that takes place every 10 years) to measure numerous features of the US population. The data we will be working with includes about 100 variables from the 2017 ACS survey, and is a 10% sample of the ACS (which itself is a 1% sample of the US population, so we're working with about a 0.1% sample of the United States). 

This data comes from [IPUMS](https://usa.ipums.org/usa/), which provides a very useful tool for getting subsets of major survey datasets, not just from the US, but [from government statistical agencies the world over](https://international.ipums.org/international-action/sample_details).

This is *real* data, meaning that you are being provided the data as it is provided by IPUMS. Documentation for all variables used in this data can be found [here](https://usa.ipums.org/usa-action/variables/group) (you can either search by variable name to figure out the meaning of a variable in this data, or search for something you want to see if a variable with the right name is in this data). 

Within this data is information on both the educational background and current earnings of a representative sample of Americans. We will now use this data to estimate the labor-market returns to graduating high school and college, and to learn something about the meaning of an educational degree. 

## Exercises

### Exercise 1

Import `US_ACS_2017_10pct_sample.dta` into a pandas DataFrame. 

This can be done with the command `pd.read_stata("US_ACS_2017_10pct_sample.dta")`, which will read in files created in the program Stata (and which uses the file suffix `.dta`). This is a format commonly used by social scientists. We will discuss the range of tools for importing data built into pandas in more detail next week!

In [1]:
import pandas as pd

df = pd.read_stata("US_ACS_2017_10pct_sample.dta")

df.head()

,year,datanum,serial,cbserial,numprec,subsamp,hhwt,hhtype,cluster,adjust,...,migcounty1,migmet131,vetdisab,diffrem,diffphys,diffmob,diffcare,diffsens,diffeye,diffhear
0,2017,1,177686,2.017001e+12,9,64,55,"female householder, no husband present",2.017002e+12,1.011189,...,0,not in identifiable area,n/a,n/a,n/a,n/a,n/a,no vision or hearing difficulty,no,no
1,2017,1,1200045,2.017001e+12,6,79,25,"male householder, no wife present",2.017012e+12,1.011189,...,0,not in identifiable area,n/a,no cognitive difficulty,no ambulatory difficulty,no independent living difficulty,no,no vision or hearing difficulty,no,no
2,2017,1,70831,2.017000e+12,1 person record,36,57,"male householder, living alone",2.017001e+12,1.011189,...,0,not in identifiable area,n/a,has cognitive difficulty,no ambulatory difficulty,no independent living difficulty,no,no vision or hearing difficulty,no,no
3,2017,1,557128,2.017001e+12,2,10,98,married-couple family household,2.017006e+12,1.011189,...,0,not in identifiable area,n/a,no cognitive difficulty,no ambulatory difficulty,no independent living difficulty,no,no vision or hearing difficulty,no,no
4,2017,1,614890,2.017001e+12,4,96,54,married-couple family household,2.017006e+12,1.011189,...,0,not in identifiable area,n/a,n/a,n/a,n/a,n/a,no vision or hearing difficulty,no,no


## Getting to Know Your Data

When you get a new dataset like this, it's good to start by trying to get a feel for its contents and organization. Toy datasets you sometimes get in classes are often very small, and easy to look at, but this is a pretty large dataset, so you can't just open it up and get a good sense of it. Here are some ways to get to know your data. 

### Exercise 2

How many observations are in your data?

In [2]:
df.shape

(319004, 104)

### Exercise 3

How many variables are in your data?

In [3]:
df.columns.nunique()

104

### Exercise 4

 Let's see what variables are in this dataset. First, try to see them all using the command:


```python
acs.columns
```

As you will see, `python` doesn't like to print out all the different variables when there are this many in a dataset. 

To get everything printed out, we can loop over all the columns and print them one at a time with the command:

```
for c in acs.columns: print(c)
```

It's definitely a bit of a hack, but honestly a pretty useful one!

In [4]:
df.columns

Index(['year', 'datanum', 'serial', 'cbserial', 'numprec', 'subsamp', 'hhwt',
       'hhtype', 'cluster', 'adjust',
       ...
       'migcounty1', 'migmet131', 'vetdisab', 'diffrem', 'diffphys', 'diffmob',
       'diffcare', 'diffsens', 'diffeye', 'diffhear'],
      dtype='object', length=104)

In [5]:
print(df.columns)

Index(['year', 'datanum', 'serial', 'cbserial', 'numprec', 'subsamp', 'hhwt',
       'hhtype', 'cluster', 'adjust',
       ...
       'migcounty1', 'migmet131', 'vetdisab', 'diffrem', 'diffphys', 'diffmob',
       'diffcare', 'diffsens', 'diffeye', 'diffhear'],
      dtype='object', length=104)


### Exercise 5

That's a *lot* of variables, and definitely more than we need. In general, life is easier when working with these kinds of huge datasets if you can narrow down the number of variables a little. In this exercise, we will be looking at the relationship between education and wages, we need variables for: 

- Age
- Income
- Education
- Employment status (is the person actually working)

These quantities of interest correspond to the following variables in our data: `age`, `inctot`, `educ`, and `empstat`. 

Subset your data to just those variables. 

In [6]:
df2 = df[['age', 'inctot', 'educ', 'empstat']]

df2

,age,inctot,educ,empstat
0,4,9999999,nursery school to grade 4,n/a
1,17,6000,grade 11,employed
2,63,6150,4 years of college,employed
3,66,14000,grade 12,not in labor force
4,1,9999999,n/a or no schooling,n/a
...,...,...,...,...
318999,33,22130,4 years of college,employed
319000,4,9999999,nursery school to grade 4,n/a
319001,20,5000,grade 12,employed
319002,47,240000,5+ years of college,employed


### Exercise 6 

Now that we have a more manageable number of variables, it's often very useful to look at a handful of rows of your data. The easiest way to do this is probably the `.head()` method (which will show you the first five rows), or the `tail()` method, which will show you the last five rows. 

But to get a good sense of your data, it's often better to use the `sample()` command, which returns a random set of rows. As the first and last rows are sometimes not representative, a random set of rows can be very helpful. Try looking at a random sample of 20 rows (note: you don't have to run `.sample()` ten times to get ten rows. Look at the `.sample` help file if you're stuck. 

In [7]:
df2.sample(20)

,age,inctot,educ,empstat
208027,84,20000,grade 12,not in labor force
228951,4,9999999,nursery school to grade 4,n/a
9567,68,12000,1 year of college,not in labor force
134579,3,9999999,n/a or no schooling,n/a
211914,53,43000,1 year of college,employed
222709,38,90000,1 year of college,employed
62304,66,41500,grade 12,not in labor force
38743,50,25000,2 years of college,employed
148675,51,10500,grade 12,not in labor force
167810,94,5200,"grade 5, 6, 7, or 8",not in labor force


### Exercise 7

Do you see any immediate problems? What issues do you see?

In [8]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 319004 entries, 0 to 319003
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype   
---  ------   --------------   -----   
 0   age      319004 non-null  category
 1   inctot   319004 non-null  int32   
 2   educ     319004 non-null  category
 3   empstat  319004 non-null  category
dtypes: category(3), int32(1)
memory usage: 4.6 MB


### Exercise 8 

One problem is that many people seem to have incomes of $9,999,999. Moreover, people with those incomes seem to be very young children. 

What you are seeing is one method (a relatively old one) for representing missing data. In this case, the value 9999999 is being used as a **sentinel value** — a way to denote missing data that was used back in the day when there was no way to add a special data type for missing data. In this case, it identifies observations where the person is too young to work, so their income value is missing. 

So let's begin by dropping anyone who has `inctot` equal to 9999999.

After dropping, how many observations do you have?

In [9]:
df3 = df2[df2['inctot'] != 9999999]
df3

,age,inctot,educ,empstat
1,17,6000,grade 11,employed
2,63,6150,4 years of college,employed
3,66,14000,grade 12,not in labor force
5,50,50000,grade 12,employed
6,82,27100,1 year of college,not in labor force
...,...,...,...,...
318998,31,0,5+ years of college,not in labor force
318999,33,22130,4 years of college,employed
319001,20,5000,grade 12,employed
319002,47,240000,5+ years of college,employed


### Exercise 9

OK, the other potential problem is that our data includes lots of people who are unemployed and people who are not in the labor force (this means they not only don't have a job, but also aren't looking for a job). For this analysis, we want to focus on the wages of people who are currently employed. So subset the dataset for the people for whom `empstat` is equal to "employed". 

Note that our decision to only look at people who are employed impacts how we should interpret the relationship we estimate between education and income. Because we are only looking at employed people, we will be estimating the relationship between education and income *for people who are employed*. That means that if education affects the *likelihood* someone is employed, we won't capture that in this analysis.

(You might also want to run `.sample()` after this just to make sure you were successful in your subsetting).

After this subsetting, how many observations do you have?

In [10]:
df4 = df3[df3['empstat'] == "employed"]
df4

,age,inctot,educ,empstat
1,17,6000,grade 11,employed
2,63,6150,4 years of college,employed
5,50,50000,grade 12,employed
9,17,2000,grade 12,employed
10,47,18000,n/a or no schooling,employed
...,...,...,...,...
318995,67,125000,grade 12,employed
318999,33,22130,4 years of college,employed
319001,20,5000,grade 12,employed
319002,47,240000,5+ years of college,employed


In [11]:
df4.columns

Index(['age', 'inctot', 'educ', 'empstat'], dtype='object')

### Exercise 10

Now let's turn to education. The `educ` variable seems to have a lot of discrete values. Let's see what values exist, and their distribution, using the `value_counts()` method. This is an *extremely* useful tool you'll use a lot! Try the following code (modified for the name of your dataset, of course):

```python
acs["educ"].value_counts()
```

In [12]:
df4["educ"].value_counts()

grade 12                     47815
4 years of college           33174
1 year of college            22899
5+ years of college          20995
2 years of college           14077
grade 11                      2747
grade 5, 6, 7, or 8           2092
grade 10                      1910
n/a or no schooling           1291
grade 9                       1290
nursery school to grade 4      468
Name: educ, dtype: int64

### Exercise 11

There are a lot of values in here, so let's just check a couple. What is the average value of `inctot` for people whose highest grade level is "grade 12" (in the US, that is someone who has graduated high school)? 

*Please round your answer to two decimal places* remembering to not round any intermediate results.

In [13]:
df5 = df4[df4["educ"] == "grade 12"]
df5

,age,inctot,educ,empstat
5,50,50000,grade 12,employed
9,17,2000,grade 12,employed
22,28,58000,grade 12,employed
23,48,52000,grade 12,employed
39,20,43000,grade 12,employed
...,...,...,...,...
318973,50,32000,grade 12,employed
318983,64,45600,grade 12,employed
318987,36,50000,grade 12,employed
318995,67,125000,grade 12,employed


In [14]:
df5["inctot"].mean()

38957.76068179442

### Exercise 12

What is the average income of someone who has completed an undergraduate degree but not done any postgraduate education ("4 years of college")? 

*Please round your answer to two decimal places* remembering to not round any intermediate results.

In [15]:
df5 = df4[df4["educ"] == "4 years of college"]
df5

,age,inctot,educ,empstat
2,63,6150,4 years of college,employed
35,49,4600,4 years of college,employed
45,25,59000,4 years of college,employed
50,52,30000,4 years of college,employed
59,30,40000,4 years of college,employed
...,...,...,...,...
318969,70,67400,4 years of college,employed
318976,61,42130,4 years of college,employed
318979,51,6500,4 years of college,employed
318982,26,45000,4 years of college,employed


In [16]:
df5["inctot"].mean()

75485.05293301983

### Exercise 13

In percentage terms, how much more does an employed college graduate earn as compared to someone who is only a high school graduate? In other words, calculate how much more an employed college graduate earns than an employed high school graduate, and divide that by the average earnings of a high school graduate.

Put your answer in percentage terms (so 0 implies they earn the same amount, and 100 implies a college graduate earns 100% more (double) what a high school graduate earns). 

*Please round your answer to one decimal place* remembering to not round any intermediate results.

In [17]:
hs_mean_income = df4[df4["educ"] == "grade 12"]["inctot"].mean()
college_mean_income = df4[df4["educ"] == "4 years of college"]["inctot"].mean()


In [18]:
percentage_increase = ((college_mean_income - hs_mean_income) / hs_mean_income) * 100


In [19]:
percentage_increase = round(percentage_increase, 1)


In [20]:
print(f"Employed college graduates earn {percentage_increase}% more than employed high school graduates on average.")


Employed college graduates earn 93.8% more than employed high school graduates on average.



### Exercise 14

What does that suggest is the value of getting a college degree after graduating high school?

### Exercise 15

What is the difference in the average income for somebody whose highest grade completed is Grade 11 and somebody whose highest grade completed is Grade 10? *Please round your answer to two decimal places.* Remember to not round any intermediate results.


In [21]:
grade_11_mean_income = df4[df4["educ"] == "grade 11"]["inctot"].mean()
grade_10_mean_income = df4[df4["educ"] == "grade 10"]["inctot"].mean()


In [22]:
income_difference = grade_11_mean_income - grade_10_mean_income

income_difference = round(income_difference, 2)

print(f"The difference in average income between Grade 11 and Grade 10 graduates is ${income_difference}.")



The difference in average income between Grade 11 and Grade 10 graduates is $-1477.11.


### Exercise 16

What is the difference in the average income for somebody whose highest grade completed is Grade 12 (has finished high school) and somebody whose highest grade completed is Grade 11?

*Please round your answer to two decimal places.* Remember to not round any intermediate results.

In [23]:
grade_12_mean_income = df4[df4["educ"] == "grade 12"]["inctot"].mean()
grade_11_mean_income = df4[df4["educ"] == "grade 11"]["inctot"].mean()

income_difference = grade_12_mean_income - grade_11_mean_income

income_difference = round(income_difference, 2)

print(f"The difference in average income between Grade 12 (high school graduates) and Grade 11 graduates is ${income_difference}.")



The difference in average income between Grade 12 (high school graduates) and Grade 11 graduates is $17416.07.


### Exercise 17

Moving from Grade 10 to Grade 11 and moving from Grade 11 to Grade 12 both constitute getting one additional year of high school education. Are these two increases in education associated with the same change in average earnings? Can you think of a reason why?


## Take-aways

Congratulations! You just discovered "the sheepskin effect!": people with degrees tend to earn substantially more than people who have *almost* as much education, but don't have an actual degree. 

In economics, this is viewed as evidence that the reason employers pay people with high school degrees more than those without degree is *not* that they think those who graduated high school have learned specific, useful skills. If that were the case, we would expect employee earnings to rise with every year of high school, since in each year of high school we learn more. 

Instead, this suggests employees pay high school graduates more because they think *the kind of people* who can finish high school are the *kind of people* who are likely to succeed at their jobs. Finishing high school, in other words, isn't about accumulating specific knowledge; it's about showing that you *are the kind of person* who can rise to the challenge of finishing high school, also suggesting you are also the kind of person who can succeed as an employee. 

(Obviously, this does not tell us whether that is an *accurate* inference, just that that seems to be how employeers think.) 

In other words, in the eyes of employers, a high school degree is a *signal* about the kind of person you are, not certification that you've learned a specific set of skills (an idea that earned [Michael Spence](https://en.wikipedia.org/wiki/Michael_Spence) a Nobel Prize in Economics). 

### Data Citation

The ACS data used in this exercise are a subsample of the IPUMS USA data available from [usa.ipums.org.](usa.ipums.org)

Please cite use of the data as follows: Steven Ruggles, Sarah Flood, Sophia Foster, Ronald Goeken, Jose Pacas, Megan Schouweiler and Matthew Sobek. IPUMS USA: Version 11.0 [dataset]. Minneapolis, MN: IPUMS, 2021. https://doi.org/10.18128/D010.V11.0

These data are intended for this exercise only. Individuals analyzing the data for other purposes must submit a separate data extract request directly via IPUMS USA.

Individuals are not to redistribute the data without permission. Contact ipums@umn.edu for redistribution requests.